# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
collections = db.list_collection_names()
print("Collections in 'uk_food' database:")
pprint(collections)

Collections in 'uk_food' database:
['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")
# Display the first document in the results using pprint
first_document = establishments.find_one(query)
print("\nFirst establishment document with a hygiene score of 20:")
pprint(first_document)

Number of establishments with a hygiene score of 20: 287

First establishment document with a hygiene score of 20:
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6733cd33bd195f66d8daacbc'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 '

In [7]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)

# Display the number of rows in the DataFrame
results_list = list(results)
df = pd.DataFrame(results_list)
# Display the first 10 rows of the DataFrame
print("\nFirst 10 rows of the DataFrame:")
print(df.head(10))


First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  6733cd33bd195f66d8daacbc   110681                  0   
1  6733cd33bd195f66d8dab03d   612039                  0   
2  6733cd33bd195f66d8dab347   730933                  0   
3  6733cd33bd195f66d8dab535   172735                  0   
4  6733cd33bd195f66d8dab544   172953                  0   
5  6733cd33bd195f66d8dabee3   512854                  0   
6  6733cd33bd195f66d8dac104  1537089                  0   
7  6733cd33bd195f66d8dad62f   155648                  0   
8  6733cd33bd195f66d8dada73  1012883                  0   
9  6733cd33bd195f66d8dae285   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532      

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [21]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": "London",   # Filtering for London
    "RatingValue": {"$gte": 4}        # RatingValue greater than or equal to 4
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {count}")
# Display the first document in the results using pprint
first_document = establishments.find_one(query)
print("\nFirst establishment document with RatingValue >= 4 in London:")
pprint(first_document)

Number of establishments in London with RatingValue >= 4: 0

First establishment document with RatingValue >= 4 in London:
None


In [22]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(f"\nNumber of rows in the DataFrame: {len(df)}")
# Display the first 10 rows of the DataFrame
print("\nFirst 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 1

First 10 rows of the DataFrame:
    _id   count
0  None  117789


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [16]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    "Latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "Longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5  # Ensures RatingValue is 5
}
sort = [("scores.Hygiene", -1)]  # Use -1 for descending order (highest hygiene score first)

limit = 10  

# Perform the query with sorting and limiting
results = establishments.find(query).sort(sort).limit(limit)

# Print the results
print(f"\nFound {limit} establishments with RatingValue 5, within {degree_search} degree of the given coordinates:")

# Iterate through the results and print each one
for establishment in results:
    pprint(establishment)



Found 10 establishments with RatingValue 5, within 0.01 degree of the given coordinates:


In [17]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(results)

# Display the DataFrame
print("\nDataFrame of query results:")
print(df)


DataFrame of query results:
Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [18]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results
# Define the aggregation pipeline
pipeline = [
    {"$match": {"scores.Hygiene": 0}},  # Step 1: Match establishments with hygiene score of 0
    {"$group": {"_id": "$LocalAuthority", "count": {"$sum": 1}}},  # Step 2: Group by LocalAuthority
    {"$sort": {"count": -1}}  # Step 3: Sort by count in descending order
]

# Execute the pipeline
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of Local Authorities with establishments having a hygiene score of 0: {len(results)}")

# Print the first 10 results
print("\nFirst 10 Local Authorities with the highest number of establishments with a hygiene score of 0:")
for result in results[:10]:
    pprint(result)

Number of Local Authorities with establishments having a hygiene score of 0: 1

First 10 Local Authorities with the highest number of establishments with a hygiene score of 0:
{'_id': None, 'count': 117789}


In [20]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Rename '_id' column to 'LocalAuthority' for clarity
df = df.rename(columns={"_id": "LocalAuthority"})

# Display the number of rows in the DataFrame
print(f"\nNumber of rows in the DataFrame: {len(df)}")
# Display the first 10 rows of the DataFrame
print("\nFirst 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 1

First 10 rows of the DataFrame:
  LocalAuthority   count
0           None  117789
